### TOPUP 

1. acceder al arbol de directorio
2. recuperar los datos que son importantes: echotime totalreadtime (acqparameters.txt)
3. coger solo la primera imagen B0
4. brain mask
5. subprocess

In [4]:
import os
import nipype
import json
from pathlib import Path

200219-11:11:36,968 nipype.utils INFO:
	 No new version available.


In [3]:
#path="C:\Users\viole\Desktop\TFG\SENECA-PICASO"

In [4]:
#camino = Path(main+'Documentos/archivos_texto/')

In [5]:
sep=os.path.sep

main= 'C:'+sep+'Users'+ sep+'viole'+sep+'Desktop'+sep+'TFG'+sep+'SENECA-PICASO'

In [6]:
os.getcwd()
os.listdir("nifti\\nifti\\sub-sp01")

['ses-01', 'ses-02']

In [7]:
rootDir=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'func'+sep+'sub-sp01'+sep+'ses-02_task-rest_bold.json'

for dirName, subdirList, fileList in os.walk(rootDir, topdown=False):
    print('hola')
    print('Found directory: %s' % fileList)

In [8]:

os.access("nifti", os.R_OK) 

True

1. Read the json files 

In [9]:
file=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'func'+sep+'sub-sp01ses-02_task-rest_bold.json'


In [10]:
with open(file) as json_file:
    data = json.load(json_file)
    EES=data["EffectiveEchoSpacing"]
    AdqMatPE=data["AcquisitionMatrixPE"]
    ET=data["EchoTime"]
    PhaseDir=data["PhaseEncodingDirection"]
    print(EES, AdqMatPE, ET, PhaseDir)
    

0.00078 96 0.0248 j-


2. Calculate TotalReadOutTime

In [11]:
ReadOut= 0.5*EES*AdqMatPE

In [12]:
ReadOut

0.03744

3. Creat acqparameters.txt

In [13]:
string=PhaseDir
f= open(os.path.join(main,"acqparameters.txt"),"w+")
f.write('0'+' '+string.replace('j-', '-1').replace('j','1')+' '+'0'+' '+str(ReadOut)) 
f.close()

4. Make comandos 

In [19]:
#fslroi sub-sp01ses-02_dir-AP_dwi.nii nodif 0 1
#sub-sp01ses-02_dir-AP_dwi.nii

In [15]:
from nipype.interfaces.fsl import ExtractROI
dwidata=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'dwi'+sep+'sub-sp01ses-02_dir-AP_dwi.nii'
fslroi = ExtractROI(in_file=dwidata, roi_file='nodif.nii', t_min=0,t_size=1)#como llamabamos el nodif
fslroi.cmdline == 'fslroi %s nodif.nii 0 1' % dwidata

'hi'


In [18]:
#fls merge  -t AP_PA_b0 nodif nodif_PA
#C:\Users\viole\Desktop\TFG\SENECA-PICASO\nifti\nifti\sub-sp01\ses-02

In [ ]:
from nipype.interface import fsl
dataIn=dwidata=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'AP_PA_b0.nii.gz'
nodif=dwidata=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+
fsl_merge = mem.cache(fsl.Merge)
results = fsl_merge(dimension='t', in_files=[dataIn, nodif])
results.outputs.merged_file

In [17]:
#topup --imain=AP_PA_b0 --datain=acqparams.txt --config=b02b0.cnf --out=topup_AP_PA_b0
#imain=AP_PA_b0.nii.gz
#datain=acqparams.txt
#config=b02b0.cnf(?)
#out=topup_AP_PA_b0

In [ ]:
from nipype.interfaces.fsl import ApplyTOPUP
applytopup = ApplyTOPUP()
imain=main+sep+'nifti'+sep+'nifti'+sep+'sub-sp01'+sep+'ses-02'+sep+'AP_PA_b0.nii.gz'
applytopup.inputs.in_files = [imain]
applytopup.inputs.encoding_file = "acqparams.txt"
applytopup.inputs.in_topup_fieldcoef = "topup_fieldcoef.nii.gz"#de donde ha salido
applytopup.inputs.in_topup_movpar = "topup_movpar.txt"#de donde ha salido
applytopup.inputs.output_type = "NIFTI_GZ"
applytopup.cmdline 
res = applytopup.run() 